## Draft metrics of dataset value

In [1]:
from server import * # to do: clean up and rename 'server' script

In [2]:
# build a graph from the JSON-LD corpus
net = RCNetwork()
# net.parse_corpus(Path("../corpus/corpus.jsonld"))

In [ ]:
# rank and scale each entity
# net.build_analytics_graph()

In [3]:
# parses, builds NetworkX graph, and creates default "rank" for each entity
net.load_network("../../rclc/corpus.jsonld")

6584.664106369019

#### simple ways to explore the RCNetwork

In [ ]:
# view an element of the graph
net.data['dataset-b1b062c8a276fe2800fd'].view

In [ ]:
# net.prov['provider-99fda6d9f6fbf67c3f72'].view

net.prov['provider-99fda6d9f6fbf67c3f72'].view['title']

In [ ]:
# net.get_id('publication-639196e25a9adcadaaf2')
# net.id_list # returns

net.get_id('dataset-b1b062c8a276fe2800fd') # to find in nxg if needed

In [26]:
len(net.publ.keys()), len(net.prov.keys()), len(net.data.keys())

(1604, 310, 208)

In [ ]:
subgraph, paths, node_id = net.get_subgraph('NOAA', 2)

In [ ]:
# no entries with full NOAA name
for p in net.prov.keys():
    if net.prov[p].view['title']=='National Oceanic and Atmospheric Administration':
        print(net.prov[p].view)

In [4]:
# provider entry uses abbreviation
for p in net.prov.keys():
    if net.prov[p].view['title']=='NOAA':
        print(net.prov[p].view)

{'id': 'provider-e4db130cccc74f809faf', 'title': 'NOAA', 'ror': 'https://ror.org/02z5nhe81', 'used': True}


In [5]:
## gather NOAA datasets

data_list = []

for d in net.data.keys():
    if net.data[d].view['provider']=='provider-e4db130cccc74f809faf':
#         print(net.data[d].view)
        data_list.append(net.data[d].view['id'])

### some more ways to access RCNetwork

In [ ]:
net.data[data_list[3]].view

In [ ]:
# edges from this node
net.nxg[net.get_id(net.data[data_list[3]].view["id"])]

In [ ]:
net.id_list[2617]

In [ ]:
net.labels[25501]

In [ ]:
# test 'recommended links' function
uuid, title, rank, url, provider, publ_list = net.reco_data(net.data[data_list[3]])

In [ ]:
for pub in net.publ.keys():
    if 'dataset-26993827b8a6f43da2bc' in net.publ[pub].view['datasets']:
        print(net.publ[pub].view)

In [ ]:
for pub in publ_list:
    print(pub)

In [ ]:
net.publ[net.id_list[25501]].view

In [ ]:
# spot check a dataset in the list is found in at least some publications
for p in net.publ.keys():
        if data_list[4] in net.publ[p].view['datasets']:
            print('found data')

### Gather dataset labels and initial measures

In [21]:
DataID = [] # RC UUID
DataName = [] # title
DataRank = [] # rank
RecPubCount = [] # count of publications from 'recommender'
TotPubCount = [] # total publications that are linked to this dataset
CombinedData = [] # total count of other datasets linked to publications
UniqueCombData = [] # unique count of other datsets linked to publications

for d in data_list:
    # gather  info from 'recommender'; it sorts the publications based on Eigenvector calc of 'rank'
    uuid, title, rank, url, provider, publ_list = net.reco_data(net.data[d])
    DataID.append(uuid)
    DataName.append(title)
    DataRank.append(rank)
    RecPubCount.append(len(publ_list))
    # search through all publications for current dataset
    co_data = []
    num_pubs = 0
    for p in net.publ.keys():
        if d in net.publ[p].view['datasets']:
            num_pubs += 1 # pub uses current data
            d_list = net.publ[p].view['datasets'].copy()
            d_list.remove(uuid) # don't include current dataset
            co_data = co_data + d_list # ** LIST OF DATASETS; use to eg calculate data provider co-occurence
            #print('----- added pub')
    TotPubCount.append(num_pubs)
    CombinedData.append(len(co_data))
    UniqueCombData.append(len(set(co_data)))
    
# save metrics to dataframe to explore
df = pd.DataFrame({'RC_uuid': DataID, 'Dataset_Name': DataName, 'RC_rank': DataRank, 'num_rec_pubs': RecPubCount,
                  'total_pubs': TotPubCount, 'combined_data': CombinedData, 'num_unique_data_join': UniqueCombData})

In [22]:
df

,RC_uuid,Dataset_Name,RC_rank,num_rec_pubs,total_pubs,combined_data,num_unique_data_join
0,dataset-51708b6d1d43040d0085,Global Historical Climatology Network,0.2528,1,1,0,0
1,dataset-df0ad972c8f854896ac4,Global Forecast System,0.5785,24,24,0,0
2,dataset-e5e638ae2642a22ce7a1,Integrated Surface Database,0.4353,15,15,0,0
3,dataset-26993827b8a6f43da2bc,Coastal Topographic Lidar,0.2514,1,1,2,2
4,dataset-fb48d7d456ee58f91186,Sea Level Rise Inundation,0.2541,2,2,3,3
5,dataset-ecaf5e384d1ecc8723c8,National Data Buoy Center,0.3278,3,3,4,4
6,dataset-9c5fed76bbcb0ce5d978,Economics: National Ocean Watch,0.3187,5,5,6,5
7,dataset-05c58f6caecd4f6d57ff,Long Range Forecasts,0.2520,2,2,1,1
8,dataset-545bdd82bd282357400b,Tidal Datum,0.3012,4,4,1,1
9,dataset-552af54ec1572ce1c5bc,"Sea, Lake, and Overland Surges from Hurricanes",0.2537,1,1,2,2


In [23]:
# save metrics as table
df.to_csv('rc_metrics_0915.csv', index=False)

### data checking

In [ ]:
## spot check pub list returned by recommender
for publist in net.reco_data(net.data['dataset-f513907681e789b16587'])[5]:
    pub = net.publ[net.id_list[publist[0]]]
    print(pub.view['title'])
    print('---', pub.view['datasets'])

In [ ]:
## confirm same publications returned
for pkey in net.publ.keys():
    if 'dataset-f513907681e789b16587' in net.publ[pkey].view['datasets']:
        print(net.publ[pkey].view['title'])
        print('--',net.publ[pkey].view['datasets'])

In [ ]:
for publist in net.reco_data(net.data['dataset-545bdd82bd282357400b'])[5]:
    pub = net.publ[net.id_list[publist[0]]]
    print(pub.view['title'])
    print('---', pub.view['datasets'])

In [ ]:
for pkey in net.publ.keys():
    if 'dataset-545bdd82bd282357400b' in net.publ[pkey].view['datasets']:
        print(net.publ[pkey].view['title'])
        print('--',net.publ[pkey].view['datasets'])

In [ ]:
for pkey in net.publ.keys():
    if 'dataset-df0ad972c8f854896ac4' in net.publ[pkey].view['datasets']:
        print(net.publ[pkey].view['title'])
        print('--',net.publ[pkey].view['datasets'])


In [24]:
for pkey in net.publ.keys():
    if 'dataset-df0ad972c8f854896ac4' in net.publ[pkey].view['datasets']:
        print(net.publ[pkey].view['title'])
        print('--',net.publ[pkey].view['datasets'])

A Case Study of Low-Level Jets in Yerevan Simulated by the WRF Model
-- ['dataset-df0ad972c8f854896ac4']
Study on Wind Simulations Using Deep Learning Techniques during Typhoons: A Case Study of Northern Taiwan
-- ['dataset-df0ad972c8f854896ac4']
Extreme Weather and Poverty Risk: Evidence from Multiple Shocks in Mozambique
-- ['dataset-df0ad972c8f854896ac4']
IMPLEMENTATION OF A COMPREHENSIVE AND EFFECTIVE GEOPROCESSING WORKFLOW ENVIRONMENT
-- ['dataset-df0ad972c8f854896ac4']
Advanced methods for uncertainty assessment and global sensitivity analysis of a Eulerian atmospheric chemistry transport model
-- ['dataset-df0ad972c8f854896ac4']
High-Resolution Discharge Forecasting for Snowmelt and Rainfall Mixed Events
-- ['dataset-df0ad972c8f854896ac4']
A top-down assessment using OMI NO2 suggests an underestimate in the NOx emissions inventory in Seoul, South Korea during KORUS-AQ
-- ['dataset-df0ad972c8f854896ac4']
Impacts of North Pacific subtropical and subarctic oceanic frontal zones on 